In [1]:
class Ball:
    def __init__(self, frame=None, x=None, y=None):
        self._frame = frame
        self._x = x
        self._y = y
    
    @property
    def frame(self):
        return self._frame
    
    @frame.setter
    def frame(self, value):
        self._frame = value
    
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self._x = value
    
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self._y = value

In [2]:
#Line的成員有 start、end、m、way
#分別對應 起始球(物件)、終點球(物件)、斜率、線的方向

class Line:
    def __init__(self,start=None,end=None,m=None,way=None): #way主要來判斷兩球變化方向 0為向上 1為向下 2為dont care
        self._start = start
        self._end = end
        try:
            self._m = (self._start.x-self._end.x)/(self._start.y - self._end.y)
        except:
            self._m = 0
        if abs(start.y - end.y)/(end.frame-start.frame) < 0.0235: #判斷y如果位移小於0.0235則為don'tcare
            self._way = 2
        #判斷X如果位移小於0.02則為dontcare
        elif abs(start.x - end.x)/(end.frame-start.frame) < 0.1 and abs(start.y - end.y)/(end.frame-start.frame) < 0.02:
            self._way = 2
        elif start.y > end.y: #0向上飛
            self._way = 0
        else:               #1向下飛
            self._way = 1
    
    @property
    def start(self):
        return self._start
        
    @start.setter
    def start(self,value):
        self._start = value
    
    @property
    def end(self):
        return self._end
        
    @end.setter
    def end(self,value):
        self._end = value
    
    @property
    def m(self):
        return self._m
        
    @m.setter
    def m(self,value):
        self._m = value
        
    @property
    def way(self):
        return self._way
        
    @way.setter
    def way(self,value):
        self._way = value

In [3]:
#wave的成員有 balls、start、end、highest、d_y
#分別對應 拋物線中的球(list)、起始球(物件)、終點球(物件)、最高點(索引)、最高點與起始球的 Y值差
class Wave:
    def __init__(self,balls=[],highest=None,start=None,end=None,d_y=None):
        self._balls = balls #ball classes
        self._highest = highest #ball index
        self._start = start #ball class
        self._end = end #ball class
        self._d_y = d_y #ball class
        
        if self._highest == None: #設定最高點
            self._highest = 0
            for i in range(len(self._balls)):
                if(self._balls[i].y < self._balls[self._highest].y):
                    self._highest = i
        
        if self._start == None: #設定wave的起始球點
            self._start = self._balls[0]
        
        if self._end == None: #設定wave的結束球點
            self._end = self._balls[-1]
        
        #下一行是波的起始球點y值與最高球點相減
        if self._start != None and self._d_y == None : 
            self._d_y = abs(balls[self._highest].y - self._start.y) #/abs(waves[i].balls[highest].frame - waves[i].start.frame)
   
    @property
    def balls(self):
         return self._balls
    
    @balls.setter
    def balls(self,value):
        self._balls = value
        
    @property
    def highest(self):
        return self._highest
        
    @highest.setter
    def highest(self,value):
        self._highest = value
        
    @property
    def start(self):
        return self._start
        
    @start.setter
    def start(self,value):
        self._start = value
    
    @property
    def end(self):
        return self._end
        
    @end.setter
    def end(self,value):
        self._end = value
    
    @property
    def d_y(self):
         return self._d_y
    
    @d_y.setter
    def d_y(self,value):
        self._d_y = value

def angle(v1, v2): #輸出角度 #參數為v1(x1,y1,x2,y2) v2(x2,y2,x3,y3)  **向量(起始x點,起始y點,終點x點,終點y點)**
    dx1 = v1[2] - v1[0]
    dy1 = v1[3] - v1[1]
    dx2 = v2[2] - v2[0]
    dy2 = v2[3] - v2[1]
    angle1 = math.atan2(dy1, dx1)
    angle1 = int(angle1 * 180/math.pi)
    # print(angle1)
    angle2 = math.atan2(dy2, dx2)
    angle2 = int(angle2 * 180/math.pi)
    # print(angle2)
    if angle1*angle2 >= 0:
        included_angle = abs(angle1-angle2)
    else:
        included_angle = abs(angle1) + abs(angle2)
        if included_angle > 180:
            included_angle = 360 - included_angle
    return included_angle

def dis(ax, ay, bx, by, ballx, bally): #計算球與人的最短距離與離球最近的人
    d_A = math.sqrt((ax-ballx)**2 + (ay-bally)**2)
    d_B = math.sqrt((bx-ballx)**2 + (by-bally)**2)
    
    if d_A > d_B:
        return ['B',d_B]
    return ['A',d_A]

def count(x, y, ballx, bally):
    d = math.sqrt((x-ballx)**2 + (y-bally)**2)
    return d

In [1]:
import csv
import cv2
import numpy as np
import time
import torch
import os
import math
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from models.experimental import attempt_load
from utils.torch_utils import select_device
from torchvision import transforms
import random
from ultralytics import YOLO

device = select_device('0')

model1 = YOLO("weights/best.pt")
model2 = attempt_load("weights/yolov7-w6-pose.pt", map_location=device)

Fusing layers... 


C:\Users\ouche\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
rows = []
with open("C:/Users/ouche/Desktop/ans.csv", 'w', newline='') as csvfile:
    with open("C:/Users/ouche/Desktop/ans.csv", 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
csvfile.close()
with open("C:/Users/ouche/Desktop/ans.csv", 'w', newline='') as csvfile:
    if(len(rows) == 0):
        writer = csv.writer(csvfile)

        # 寫入第一列(欄位)
        writer.writerow(['VideoName', 'ShotSeq', 'HitFrame', 'Hitter', 'RoundHead', 'Backhand', 'BallHeight', 'LandingX', 'LandingY', 'HitterLocationX', 'HitterLocationY', 'DefenderLocationX', 'DefenderLocationY', 'BallType', 'Winner'])
csvfile.close()
# print(rows)
rows = []

In [6]:
video_index = 1
while video_index <= 399:
    if video_index >= 170:
        videopath = f"data/part2/test/00{video_index}/00{video_index}.mp4"
        videoname = f'00{video_index}.mp4'
    else:
        if video_index < 10:
            videopath = f"data/part1/val/0000{video_index}/0000{video_index}.mp4"
            videoname = f'0000{video_index}.mp4'
        elif video_index < 100:
            videopath = f"data/part1/val/000{video_index}/000{video_index}.mp4"
            videoname = f'000{video_index}.mp4'
        else:
            videopath = f"data/part1/val/00{video_index}/00{video_index}.mp4"
            videoname = f'00{video_index}.mp4'
    
    


    #########################################################################################################################
    
    #把所有偵測到球的幀數輸入ball
    cap = cv2.VideoCapture(videopath)
    i = 0

    total = dict() #total[幀數] = [[ball_x, ball_y], [A_X, A_Y], [B_X, B_Y], [A_H, A_W], [B_H, B_W]]

    while True:


        ret, frame = cap.read()

        if not ret:
            break 

        results = model1(frame)    
        result = results[0]

        ball_index = []
        people_index = []
        if 2.0 in result.boxes.cls.tolist():#如果此幀數內有球  
            for index, value in enumerate(result.boxes.cls.tolist()):
                if value == 2.0:#ball
                    ball_index.append(index)
                if value == 0.0:#people
                    people_index.append(index)


            if len(people_index) != 2:
                i+=1
                continue

            #找一張畫面中信心度最高的球
            max = result.boxes.conf.tolist()[ball_index[0]]
            finalIndex = ball_index[0]
            for k in range(len(ball_index)): 
                if max < result.boxes.conf.tolist()[ball_index[k]]:
                    max = result.boxes.conf.tolist()[ball_index[k]]   
                    finalIndex = ball_index[k]

            #信心度大於0.5才放進去當作預測到球
            if(result.boxes.conf.tolist()[finalIndex] > 0.5): 
                ball = [result.boxes.xywh.tolist()[finalIndex]]

                #抓取人的位置
                if result.boxes.xyxy.tolist()[people_index[0]][1] > result.boxes.xyxy.tolist()[people_index[1]][1]:

                    center_B = [result.boxes.xywh.tolist()[people_index[0]][0], result.boxes.xywh.tolist()[people_index[0]][1]]#(center_B_X, center_B_Y)
                    wh_B = [result.boxes.xywh.tolist()[people_index[0]][2], result.boxes.xywh.tolist()[people_index[0]][3]]#(W_B, H_B)]
                    center_A = [result.boxes.xywh.tolist()[people_index[1]][0], result.boxes.xywh.tolist()[people_index[1]][1]]#(center_B_X, center_B_Y)
                    wh_A = [result.boxes.xywh.tolist()[people_index[1]][2], result.boxes.xywh.tolist()[people_index[1]][3]]#(W_A, H_A)]
                    
                else:
    
                    center_A = [result.boxes.xywh.tolist()[people_index[0]][0], result.boxes.xywh.tolist()[people_index[0]][1]]#(center_B_X, center_B_Y)
                    wh_A = [result.boxes.xywh.tolist()[people_index[0]][2], result.boxes.xywh.tolist()[people_index[0]][3]]#(W_A, H_A)]
                    center_B = [result.boxes.xywh.tolist()[people_index[1]][0], result.boxes.xywh.tolist()[people_index[1]][1]]#(center_B_X, center_B_Y)
                    wh_B = [result.boxes.xywh.tolist()[people_index[1]][2], result.boxes.xywh.tolist()[people_index[1]][3]]#(W_B, H_B)]
                    
                total[i] = [ball[0][:2], center_A, center_B, wh_A, wh_B] 

        if cv2.waitKey(5) & 0xFF == 27:
            break
        i+=1

    cap.release()
    cv2.destroyAllWindows()
        
    #########################################################################################################################

    #將物件Ball存進list裡
    #兩相近球形成一Line物件
    
    #total[幀數] = [[ball_x, ball_y], [A_X, A_Y], [B_X, B_Y], [A_H, A_W], [B_H, B_W]]

    balls = []
    for j in total.keys():
        balls.append([j, total[j][0]]) #balls[j] 裡面會放 [幀數, [ball_x, ball_y]] 
    
    Balls = []
    Lines = []

    for ball in balls:
        Balls.append(Ball(ball[0],ball[1][0],ball[1][1])) #轉成物件Ball形式放入Balls這個list中 

    for i in range(1,len(Balls)):
        Lines.append(Line(Balls[i-1],Balls[i])) #將兩球點放入物件Line中，形成一條線，並計算way方向 0, 1, 2
        
    #########################################################################################################################
    
    #球的處理
    #0向上飛,1向下飛
    HitFrame = []
    waves = [] #全部的波
    temp_balls = []  #一個波中所有球的資訊

    i = 0
    while i < len(Lines):
        temp_balls = [] #一個波中所有球的資訊(每波重製)
        temp_lines = [] #00000000011111111111
        if Lines[i].way == 0:  #判斷每一波的起始(球往上飛)
            
            while Lines[i].way == 0 or Lines[i].way == 2: #判斷每一波中的(球往上飛)  or dontcare              
                if Lines[i].way == 0: #此波只放入為0的line，若為dontcare則不放入。
                    temp_balls.append(Lines[i].start)
                    temp_lines.append(Lines[i].way)

                if(i+1 == len(Lines)): #讀到最後一球則跳出
                    break
                i+=1

            if(i+1 == len(Lines)): #讀到最後一球則提早跳出
                    break

            while Lines[i].way == 1 or Lines[i].way == 2: #判斷每一波的結束(球往下飛) or dontcare
                
                if Lines[i].way == 1:#此波只放入為1的line，若為dontcare則不放入。
                    temp_balls.append(Lines[i].start)
                    temp_lines.append(Lines[i].way)

                if(i+1 == len(Lines)): #讀到最後一球則跳出
                    break
                i+=1
                if Lines[i].way == 0: #1轉0則判斷為下一波開始
                    break
            
            #如果一個拋物線中，向上0 以及 向下1 的數量都沒有大於3條線，則不納入考量。
            if temp_lines.count(0) >= 3 and temp_lines.count(1) >= 3:
                wave = Wave(temp_balls) #將球加入到一個拋物線(物件Wave)中
                waves.append(wave) #將每個拋物線以waves這個list記錄下來
            print("make wave end")
        else:
            i+=1
    
    
    #########################################################################################################################
    
    wave_side = [] #拋物線兩端擊球點

    for i in range(len(waves)): #從所有波中判斷球

        highest = waves[i].highest #存下每一波的最高球點

        #wave的成員有 balls、start、end、highest、d_y
        #分別對應 拋物線中的球(list)、起始球(物件)、終點球(物件)、最高點(索引)、最高點與起始球的 Y值差

        #設定一個拋物線的y軸位移閥值
        if waves[i].d_y > 9.5: #波中的最高點與起始點差距 > 9.5
            wave_side.append(waves[i].balls[0].frame)
            wave_side.append(waves[i].balls[-1].frame)

    HitFrame = HitFrame + wave_side
    HitFrame = list(set(HitFrame)) #刪除list中重複元素
    HitFrame.sort()

    #########################################################################################################################
    
    #  殺球: 拋物線中判斷，A打向B(向下打) 或 B打向A(向上打) ，B打向A:判斷為拋物線兩端擊球點，A打向B:另外判斷
    #   1.(此幀球與下三幀球的Y差值除以幀數差值) 大於 (上三幀球與此幀球的Y差值除以幀數差值) 的2.4倍
    #   2.下一幀球的Y值處於50到240以內
    #   3.下一幀數扣掉此幀小於5
    #   4.(下三幀球的Y值) 扣掉 (下兩幀球的Y值) 大於30

    kill_ball = []

    #判斷殺球
    for i in range(len(waves)):#i為每拋物線的index
        for j in range(1,len(waves[i].balls)-1):#j為此拋物線中球的index
            highest = waves[i].highest#設定highest為此拋物線最高點index
            
            try:        
                if j == highest:#如果j是最高點，在最高點殺球     

                    #1.(此幀球與下三幀球的Y差值除以幀數差值)為 (上三幀球與此幀球的Y差值除以幀數差值) 的2.4倍 #意即突然加速度
                    if (abs(waves[i].balls[highest+3].y-waves[i].balls[highest].y)/abs(waves[i].balls[highest+3].frame-waves[i].balls[highest].frame))/(abs(waves[i].balls[highest].y-waves[i].balls[highest-3].y)/abs(waves[i].balls[highest].frame-waves[i].balls[highest-3].frame)) > 2.0:
                        print(f'加入殺球:{waves[i].balls[highest].frame}')        
                        kill_ball.append(waves[i].balls[highest].frame)

                else:#不是最高點則都要判斷，有可能為下落後才殺
                    
                    #判斷兩球幀位移方向相同
                    if (waves[i].balls[j+1].y-waves[i].balls[j].y) > 0 and (waves[i].balls[j].y-waves[i].balls[j-1].y) > 0:
                        #(此幀球與下三幀球的Y差值除以幀數差值)為 (上三幀球與此幀球的Y差值除以幀數差值) 的2.0倍 #意即下落途中突然加速度
                        if (abs(waves[i].balls[j+3].y-waves[i].balls[j].y)/abs(waves[i].balls[j+3].frame-waves[i].balls[j].frame))/(abs(waves[i].balls[j].y-waves[i].balls[j-3].y)/abs(waves[i].balls[j].frame-waves[i].balls[j-3].frame)) > 2.0:
                            
                            #因為相較於上方if時以最高點判斷，此較不嚴謹，因此加上以下條件。
                            #2.下一幀球的Y值處於50到240以內
                            if waves[i].balls[j+1].y>50 and waves[i].balls[j+1].y<240:

                                #3.下一幀數扣掉此幀小於5
                                if waves[i].balls[j+1].frame - waves[i].balls[j].frame < 5:

                                #4.(下三幀球的Y值) 扣掉 (下兩幀球的Y值) 大於30
                                    if (waves[i].balls[j+3].y-waves[i].balls[j+2].y)/(waves[i].balls[j+3].frame-waves[i].balls[j+2].frame) > 30:
                                        print(f'加入殺球:{waves[i].balls[j].frame}')        
                                        kill_ball.append(waves[i].balls[j].frame)
            except:
                continue

    print(f'經過殺球判定後')
    HitFrame = HitFrame + kill_ball
    HitFrame = list(set(HitFrame)) #刪除list中重複元素
    HitFrame.sort()
    print('結果',HitFrame) 
    print()
    
    #########################################################################################################################
   
    little_ball = []

    # 網前小球
    #   1.上一幀球的Y值與下一幀球的Y值會一起大於或一起小於此幀球的Y值，或是此幀數扣掉上一幀數大於5
    #   2.下一幀球的Y值扣掉此幀球Y值，再除以幀數差，介於兩閾值間
    #   3.上一幀球的位置與上兩幀球的位置的相差距離，再除以幀數差，介於兩閾值間

    i,j,k = 1,0,0
    while i < len(Balls):
        j = 0
        while j < len(waves):
            k = 0
            while k < len(waves[j].balls):
                if waves[j].balls[k].frame == Balls[i].frame:

                    # 1.此幀數扣掉上一幀數大於5
                    if Balls[i].frame - Balls[i-1].frame > 5:

                        #2.下一幀球的Y值扣掉此幀球得Y值，再除以幀數差，介於兩閾值間
                        y_d_f = abs(Balls[i].y - Balls[i+1].y)/abs(Balls[i].frame - Balls[i+1].frame)
                        
                        if y_d_f > 3 and y_d_f < 5:
                            print(f'加入小球1:{Balls[i].frame},2:{y_d_f}')
                            little_ball.append(Balls[i].frame)
                        elif y_d_f > 22 and y_d_f < 30:
                            print(f'加入小球1:{Balls[i].frame},2:{y_d_f}')
                            little_ball.append(Balls[i].frame)

                    #1.上一幀球的Y值與下一幀球的Y值會一起大於此幀球的Y值
                    elif Balls[i-1].y > Balls[i].y and Balls[i+1].y > Balls[i].y:

                        #2.下一幀球的Y值扣掉此幀球得Y值，再除以幀數差，介於兩閾值間
                        y_d_f = abs(Balls[i].y - Balls[i+1].y)/abs(Balls[i].frame - Balls[i+1].frame)
                        
                        if y_d_f > 3 and y_d_f < 4.8:

                            d_f = abs(Balls[i-1].y - Balls[i-2].y)/abs(Balls[i-1].frame - Balls[i-2].frame)

                            #3.上一幀球的位置與上兩幀球的位置的相差距離，再除以幀數差，介於兩閾值間
                            if d_f < 0.2:
                                print(f'加入小球2:{Balls[i].frame},2:{y_d_f}')
                                little_ball.append(Balls[i].frame)


                    #1.上一幀球的Y值與下一幀球的Y值會一起小於此幀球的Y值
                    elif Balls[i-1].y < Balls[i].y and Balls[i+1].y < Balls[i].y:

                        #2.下一幀球的Y值扣掉此幀球得Y值，再除以幀數差，介於兩閾值間
                        y_d_f = abs(Balls[i].y - Balls[i+1].y)/abs(Balls[i].frame - Balls[i+1].frame)
                        
                        if y_d_f > 2.8 and y_d_f < 5.7:

                            d_f = abs(Balls[i-1].y - Balls[i-2].y)/abs(Balls[i-1].frame - Balls[i-2].frame)

                            #3.上一幀球的位置與上兩幀球的位置的相差距離，再除以幀數差，介於兩閾值間
                            if d_f < 0.2:
                                print(f'加入小球3:{Balls[i].frame},2:{y_d_f},3:{d_f}')
                                little_ball.append(Balls[i].frame)
                            elif d_f < 15:
                                if d_f > 10:
                                    print(f'加入小球3:{Balls[i].frame},2:{y_d_f},3:{d_f}')
                                    little_ball.append(Balls[i].frame)
                k+=1
            j+=1
        i+=1


    print(f'經過小球判定後')
    HitFrame = HitFrame + little_ball
    HitFrame = list(set(HitFrame)) #刪除list中重複元素
    HitFrame.sort()
    print(HitFrame)
    print()
    
    #########################################################################################################################
    
    #  切球: A打向B(向下打) 或 B打向A(向上打) B打向A:判斷為拋物線兩端擊球點 A打向B:另外判斷
    #   1.此幀球的位置與上一幀球的位置差距 (1)小於0.005 (2)小於0.05
    #   2.下一幀球的Y值與此幀球的Y值相差 (1)大於6.01小於17 (2)大於6.01
    #   3.上兩幀球到此幀球的球路向量 與 此幀球到下一幀球的球路向量 的角度為 (1)大於130小於165 (2)大於90小於130 

    cut_ball = []

    try:
        for i in range(len(waves)):
            for j in range(2,len(waves[i].balls)-2):

                #1.此幀球的位置與上一幀球的位置差距
                if abs(waves[i].balls[j-1].y-waves[i].balls[j].y)/abs(waves[i].balls[j-1].frame-waves[i].balls[j].frame)<0.005: 

                    #2.下一幀球的Y值與此幀球的Y值相差
                    if abs(waves[i].balls[j+1].y-waves[i].balls[j].y)/abs(waves[i].balls[j].frame-waves[i].balls[j+1].frame)>6.01:#5
                        if abs(waves[i].balls[j+1].y-waves[i].balls[j].y)/abs(waves[i].balls[j].frame-waves[i].balls[j+1].frame)<17:

                            x1 = waves[i].balls[j-2].x # j-2 是為了避免切球的前一幀球與此幀是幾乎重疊的球
                            y1 = waves[i].balls[j-2].y
                            x2 = waves[i].balls[j].x
                            y2 = waves[i].balls[j].y
                            x3 = waves[i].balls[j+1].x
                            y3 = waves[i].balls[j+1].y
                            v1 = [x2,y2,x1,y1]
                            v2 = [x2,y2,x3,y3]
                            angle1 = angle(v1,v2)

                            #3.上兩幀球到此幀球的球路向量 與 此幀球到下一幀球的球路向量 的角度調整
                            if angle1>130 and angle1 < 165:
                                print(f'切球1:{waves[i].balls[j].frame},1:{abs(waves[i].balls[j-1].y-waves[i].balls[j].y)/abs(waves[i].balls[j-1].frame-waves[i].balls[j].frame)},2:{abs(waves[i].balls[j+1].y-waves[i].balls[j].y)/abs(waves[i].balls[j].frame-waves[i].balls[j+1].frame)},angle1:{angle1}')
                                cut_ball.append(waves[i].balls[j].frame)

                #1.此幀球的位置與上一幀球的位置差距
                elif abs(waves[i].balls[j-1].y-waves[i].balls[j].y)/abs(waves[i].balls[j-1].frame-waves[i].balls[j].frame)<0.05: 

                    #2.下一幀球的Y值與此幀球的Y值相差
                    if abs(waves[i].balls[j+1].y-waves[i].balls[j].y)/abs(waves[i].balls[j].frame-waves[i].balls[j+1].frame)>6.01:#5

                            x1 = waves[i].balls[j-2].x   # j-2 是為了避免切球的前一幀球與此幀是幾乎重疊的球
                            y1 = waves[i].balls[j-2].y
                            x2 = waves[i].balls[j].x
                            y2 = waves[i].balls[j].y
                            x3 = waves[i].balls[j+1].x
                            y3 = waves[i].balls[j+1].y
                            v1 = [x2,y2,x1,y1]
                            v2 = [x2,y2,x3,y3]
                            angle1 = angle(v1,v2)

                            #3.上兩幀球到此幀球的球路向量 與 此幀球到下一幀球的球路向量 的角度調整
                            if angle1<130 and angle1>90:
                                print(f'切球2:{waves[i].balls[j].frame},angle:{angle1}')
                                cut_ball.append(waves[i].balls[j].frame)
                                
    except:
        pass

    print(f'經過切球判定後')
    HitFrame = HitFrame + cut_ball
    HitFrame = list(set(HitFrame)) #刪除list中重複元素
    HitFrame.sort()
    print('結果',HitFrame) 
    print()
    
    #########################################################################################################################
    #以人球距離小於閾值(280) 判定是否為擊球幀，刪除錯誤判斷。
    hit = []
   
    for h in HitFrame:
        X_A = total[h][1][0]
        Y_A = total[h][1][1]
        X_B = total[h][2][0]
        Y_B = total[h][2][1] 
        X_ball = total[h][0][0]
        Y_ball = total[h][0][1]
        d = dis(X_A, Y_A, X_B, Y_B, X_ball, Y_ball)[1]
        print(d)
        if d <= 280:
            hit.append(h)
    HitFrame = hit

    
    #########################################################################################################################
    
    #移除落地球:落地球幀與後5幀距離相差小於10，就移除此幀，或是此幀沒有後5幀也刪除
    HitFrame.sort()
    for i in range(len(HitFrame)):
        dead_ball = HitFrame[-1]
        for j in range(len(Balls)-1):
            if Balls[j].frame == dead_ball:
                check_area = 0
                dead = j
                j+=1
                
                if Balls[dead].frame + 5 > Balls[-1].frame: #避免所需刪除的落地球沒有後5幀可以檢查
                    print(f'移除落地球: {HitFrame[-1]}')
                    del HitFrame[-1]
                    break

                while j < len(Balls)-1:

                    x = (abs(Balls[dead].x - Balls[j].x)/abs(Balls[dead].frame-Balls[j].frame))**2
                    y = (abs(Balls[dead].y - Balls[j].y)/abs(Balls[dead].frame-Balls[j].frame))**2
                    d = math.sqrt(x+y)
                    print(f'{Balls[dead].frame},{Balls[j].frame},d: {d}')
                    
                    # 前後兩幀數落差太大
                    if abs(Balls[dead].frame-Balls[j].frame) > 10:
                        break
                    if d < 10: #10 -> 5                   
                        check_area +=1
                        print(check_area)
                        if check_area >= 5: #與之後的五幀判斷
                            print(f'移除落地球: {HitFrame[-1]},d: {d}')
                            del HitFrame[-1]
                            break
                    else:
                        break
                    j+=1
                    
                if j > len(Balls) - 1 and check_area >= 3:
                    print(f'移除落地球: {HitFrame[-1]},d: {d}')
                    del HitFrame[-1]
                       
                j = dead + 1

    print(f'經過移除落地球後')
    HitFrame.sort()
    print(HitFrame)
    print()

    #########################################################################################################################
    #移除相近幀: 利用兩幀的球位置與人的最短距離比較，移除距離較大者
    i = 0
    try:
        while i < len(HitFrame):

            if abs(HitFrame[i]-HitFrame[i+1]) <= 8:

                X_A1 = total[HitFrame[i]][1][0]
                Y_A1 = total[HitFrame[i]][1][1]
                X_B1 = total[HitFrame[i]][2][0]
                Y_B1 = total[HitFrame[i]][2][1] 
                X_ball1 = total[HitFrame[i]][0][0]
                Y_ball1 = total[HitFrame[i]][0][1]
                d1 = dis(X_A1, Y_A1, X_B1, Y_B1, X_ball1, Y_ball1)[1]
                
                X_A2 = total[HitFrame[i+1]][1][0]
                Y_A2 = total[HitFrame[i+1]][1][1]
                X_B2 = total[HitFrame[i+1]][2][0]
                Y_B2 = total[HitFrame[i+1]][2][1] 
                X_ball2 = total[HitFrame[i+1]][0][0]
                Y_ball2 = total[HitFrame[i+1]][0][1]
                d2 = dis(X_A2, Y_A2, X_B2, Y_B2, X_ball2, Y_ball2)[1]

                if min(d1,d2) == d2:
                    print(f'移除相近幀數球: {HitFrame[i]}')
                    del HitFrame[i]
                    #print(i)
                    i-=1
                else:
                    print(f'移除相近幀數球: {HitFrame[i+1]}')
                    del HitFrame[i+1]
                    #print(i+1)
                    i-=1

            i+=1
    except:
        pass

    print(f'經過移除相近幀數球後')
    HitFrame.sort()
    print(HitFrame)
    
    #########################################################################################################################
    #以人球距離，判定擊球者AB方，距離A較近則為A擊球，反之。
    Hitter = []
    #total[幀數] = [[ball_x, ball_y], [A_X, A_Y], [B_X, B_Y], [A_W, A_H], [B_W, B_H]]
    for i in HitFrame:
        
        center_ax = total[i][1][0]                      #center ax                    
        center_ay = total[i][1][1]                      #center ay  
        lt_ax = center_ax - (total[i][3][0]/2)          #left top ax
        lt_ay = center_ay - (total[i][3][1]/2)          #left top ay  
        lb_ax = lt_ax                                   #left bottom ax 
        lb_ay = center_ay + (total[i][3][1]/2)          #left bottom ay 
        rt_ax = center_ax + (total[i][3][0]/2)          #right top ax 
        rt_ay = lt_ay                                   #right top ay
        rb_ax = rt_ax                                   #right bottom ax
        rb_ay = lb_ay                                   #right bottom ay
        
        
        center_bx = total[i][2][0]                      #center ax                    
        center_by = total[i][2][1]                      #center ay  
        lt_bx = center_bx - (total[i][4][0]/2)          #left top ax
        lt_by = center_by - (total[i][4][1]/2)          #left top ay  
        lb_bx = lt_bx                                   #left bottom ax 
        lb_by = center_by + (total[i][4][1]/2)          #left bottom ay 
        rt_bx = center_bx + (total[i][4][0]/2)          #right top ax 
        rt_by = lt_by                                   #right top ay
        rb_bx = rt_bx                                   #right bottom ax
        rb_by = lb_by                                   #right bottom ay

        ballx = total[i][0][0]
        bally = total[i][0][1]
                            
        da = min(count(center_ax, center_ay, ballx, bally),
                 count(lt_ax, lt_ay, ballx, bally),
                 count(lb_ax, lb_ay, ballx, bally),
                 count(rt_ax, rt_ay, ballx, bally),
                 count(rb_ax, rb_ay, ballx, bally))
        
        db = min(count(center_bx, center_by, ballx, bally),
                 count(lt_bx, lt_by, ballx, bally),
                 count(lb_bx, lb_by, ballx, bally),
                 count(rt_bx, rt_by, ballx, bally),
                 count(rb_bx, rb_by, ballx, bally))
        
        if da < db:
            Hitter.append('A')
        else:
            Hitter.append('B')

    print(HitFrame)
    print(Hitter)
    #########################################################################################################################
    
    #擊球者判定修正

    if len(Hitter) > 1:
        if(Hitter[0] == 'A'):
            for i in range(1, len(Hitter)):
                if(i%2 == 0):
                    Hitter[i] = 'A'
                else:
                    Hitter[i] = 'B'
        else:
            for i in range(1, len(Hitter)):
                if(i%2 == 0):
                    Hitter[i] = 'B'
                else:
                    Hitter[i] = 'A'

    #########################################################################################################################
    #根據所偵測到的HitFrame擊球點，導入yolopose來判定其擊球者姿勢。
    cap = cv2.VideoCapture(videopath)

    i = 0

    poses = []
    HitterLocationX = []
    HitterLocationY = []
    DefenderLocationX = []
    DefenderLocationY = []
    while True:

        ret, frame = cap.read()

        if not ret:
            break 
        
        #讀取圖片，回傳擊球者kpts骨架點。
        def pose(image):
            # preprocess image
            frame_width, frame_height = int(cap.get(3)), int(cap.get(4))

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = letterbox(image, (frame_width), stride=64, auto=True)[0]
            image_ = image.copy()
            image = transforms.ToTensor()(image)
            image = torch.tensor(np.array([image.numpy()]))

            image = image.to(device)
            image = image.float()

            with torch.no_grad():
                output, _ = model2(image)

            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model2.yaml['nc'], nkpt=model2.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
            img = image[0].permute(1, 2, 0) * 255
            img = img.cpu().numpy().astype(np.uint8)

            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            for idx in range(output.shape[0]):
                kpts = output[idx, 7:].T
                #plot_skeleton_kpts(img, kpts, 3)    
            try:    
                return kpts #回傳該幀kpts
            except:
                return ['F']
            
        if i in HitFrame:

            HitFrame_index = HitFrame.index(i)

            results = model1(frame)    
            result = results[0]

            people_index = []

            for index, value in enumerate(result.boxes.cls.tolist()):
                if value == 0.0:#people
                    people_index.append(index)

            if result.boxes.xyxy.tolist()[people_index[0]][1] > result.boxes.xyxy.tolist()[people_index[1]][1]:
                B = result.boxes.xyxy.tolist()[people_index[0]]
                A = result.boxes.xyxy.tolist()[people_index[1]]
            else:
                A = result.boxes.xyxy.tolist()[people_index[0]]
                B = result.boxes.xyxy.tolist()[people_index[1]]
    
            #找出hitterxy, defenderxy 
            if Hitter[HitFrame_index] == 'B':  #(擊球者若為B，hitter:B右下角, defender:A左下角)  (若為A, hitter:A左下角,defender:B右下角)
                #抓取B位置找pose
                img_B = frame[int(B[1])-40:int(B[3])+40, int(B[0])-40:int(B[2])+40]
                
                poses.append(pose(img_B))
                #找hitter or defender location
                HitterLocationX.append(int(B[2]))
                HitterLocationY.append(int(B[3]))
                DefenderLocationX.append(int(A[0]))
                DefenderLocationY.append(int(A[3]))
            else:
                #抓取A位置找pose
                img_A = frame[int(A[1])-30:int(A[3])+30, int(A[0])-30:int(A[2])+30]
                poses.append(pose(img_A))

                #找hitter or defender location
                HitterLocationX.append(int(A[0]))
                HitterLocationY.append(int(A[3]))
                DefenderLocationX.append(int(B[2]))
                DefenderLocationY.append(int(B[3]))

        if cv2.waitKey(5) & 0xFF == 27:
            break
        i+=1

    cap.release()
    cv2.destroyAllWindows()
    
    #########################################################################################################################
    
    #判斷是否繞頭以及正反拍
    #手腕在雙肩中心左or右
    
    i = 0
    RoundHead = []
    Backhand = []
    #等碩頤給該幀誰擊球
    
    if len(poses) > 0:
        for p in poses:
            if 'F' not in p:
                midx = (p[15] + p[18]) / 2 #雙肩X中點 
                midy = p[16] + (abs(p[15] - p[18]) / 2) #左肩y+左右肩距/2
                hand_x = p[30]
                hand_y = p[31] #total[f][1][30, 31]右手腕X,y

                '''
                    繞頭 反手
                    1    2
                    2    1
                    2    2
                '''
                if Hitter[i] == 'A':
                    if hand_x >  midx and hand_y < midy:  
                        RoundHead.append(1) #判定A有繞頭
                        Backhand.append(2) #繞頭必定不反手
                    elif hand_x > midx:
                        Backhand.append(1)#判定A反手
                        RoundHead.append(2)#反手必定不繞頭
                    else:
                        Backhand.append(2)
                        RoundHead.append(2)

                else:
                    if hand_x < midx and hand_y < midy:  
                        RoundHead.append(1) #判定B有繞頭
                        Backhand.append(2) #繞頭必定不反手
                    elif hand_x < midx:
                        Backhand.append(1)#判定B反手
                        RoundHead.append(2)#反手必定不繞頭
                    else:
                        Backhand.append(2)
                        RoundHead.append(2)
            else:
                Backhand.append(2)
                RoundHead.append(2)

    if len(RoundHead) != len(HitFrame):
        for i in range(len(RoundHead),len(HitFrame)):
            Backhand.append(2)
            RoundHead.append(2)
    
    #########################################################################################################################
    
    #判斷擊球瞬間有無高過網
    #若手握拍高過肩膀，則普遍同時也高過網。
    BallHeight = []
    if len(poses) > 0:
        for p in poses:
            if 'F' not in p:
                if p[31] < p[19]:#左手腕比左肩高,則判定擊球瞬間高過網。
                    BallHeight.append(1)
                else:
                    BallHeight.append(2)
            else:
                BallHeight.append(2)
    if len(BallHeight) != len(HitFrame):
        for i in range(len(BallHeight),len(HitFrame)):
            BallHeight.append(2)
##############################################################
            
    #判斷球種
    BallType = []

    for i in HitFrame:
        HitFrame_index = HitFrame.index(i)

        if HitFrame_index == 0 and HitFrame_index+1 < len(HitFrame): #發球
            if abs(HitFrame[HitFrame_index] - HitFrame[HitFrame_index+1]) >= 37:
                BallType.append(2) #發長球2
            else:
                BallType.append(1) #發短球1
            continue    
        if HitFrame[HitFrame_index] in kill_ball: #殺球 5
            BallType.append(5)

        elif HitFrame[HitFrame_index] in little_ball: #網前小球6
            BallType.append(6)

        elif HitFrame[HitFrame_index] in cut_ball:  #切球7
            BallType.append(7)

        else: 
            if HitFrame_index+1 < len(HitFrame): #網前小球6
                if abs(HitFrame[HitFrame_index] - HitFrame[HitFrame_index+1]) <= 30:
                    BallType.append(6)
                    continue
            while True:                             #因無法判斷，故依機率隨機給值
                random_n = random.uniform(1,68)     #長球3  挑球8
                if random_n >= 1 and random_n <= 26: #長球3
                    BallType.append(3)
                    break
                elif random_n >= 27 and random_n <= 68:  #挑球8
                    BallType.append(8)
                    break
                    
        
        #利用第一球發短球判斷反手，若發短球，則必為反手。
        if BallType[0] == 1:
            Backhand[0] = 1
    
    #########################################################################################################################
    
    rows = []
    
    with open("C:/Users/ouche/Desktop/ans.csv", 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
    csvfile.close()
    with open("C:/Users/ouche/Desktop/ans.csv", 'w', newline='') as csvfile:
   
        writer = csv.writer(csvfile)
        for row in rows:
            writer.writerow(row)
            
        if len(HitFrame) > 0:
            for j in range(len(HitFrame)):
                if j == len(HitFrame) - 1:
                    if Hitter[j] == 'A':
                        writer.writerow([videoname, j+1, HitFrame[j], Hitter[j], RoundHead[j], Backhand[j], BallHeight[j], int(total[HitFrame[j]][0][0]), int(total[HitFrame[j]][0][1]), HitterLocationX[j], HitterLocationY[j], DefenderLocationX[j], DefenderLocationY[j], BallType[j], 'B'])
                    else:
                        writer.writerow([videoname, j+1, HitFrame[j], Hitter[j], RoundHead[j], Backhand[j], BallHeight[j], int(total[HitFrame[j]][0][0]), int(total[HitFrame[j]][0][1]), HitterLocationX[j], HitterLocationY[j], DefenderLocationX[j], DefenderLocationY[j], BallType[j], 'A'])
                    break
                writer.writerow([videoname, j+1, HitFrame[j], Hitter[j], RoundHead[j], Backhand[j], BallHeight[j], int(total[HitFrame[j]][0][0]), int(total[HitFrame[j]][0][1]), HitterLocationX[j], HitterLocationY[j], DefenderLocationX[j], DefenderLocationY[j], BallType[j], 'X'])
        else:
            writer.writerow([videoname, j+1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'X']) #default輸出    

    csvfile.close()
    rows = []
    
    video_index+=1
    


0: 736x1280 2 peoples, 126.8ms
Speed: 1.0ms preprocess, 126.8ms inference, 5.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 121.0ms
Speed: 1.0ms preprocess, 121.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 122.0ms
Speed: 1.0ms preprocess, 122.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 120.0ms
Speed: 1.0ms preprocess, 120.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 120.0ms
Speed: 1.0ms preprocess, 120.0ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 120.9ms
Speed: 1.0ms preprocess, 120.9ms inference, 1.6ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 122.6ms
Speed: 1.0ms preprocess, 122.6ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 121.0ms
Speed: 1.0ms preprocess, 121.0ms inf


0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 2.0ms preprocess, 122.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 121.8ms
Speed: 1.0ms preprocess, 121.8ms inference, 1.5ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 1.0ms preprocess, 122.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 1.0ms preprocess, 122.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peopl

Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.8ms
Speed: 1.0ms preprocess, 122.8ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.3ms
Speed: 1.0ms preprocess, 124.3ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.8ms
Speed: 1.0ms preprocess, 123.8ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.2ms
Speed: 1.0ms preprocess, 123.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 1.0ms preprocess, 122.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 121.0ms
Speed: 2.0ms preproc


0: 736x1280 2 peoples, 1 ball, 124.5ms
Speed: 1.0ms preprocess, 124.5ms inference, 1.6ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 1.0ms preprocess, 122.0ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.1ms
Speed: 1.0ms preprocess, 124.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peopl


0: 736x1280 2 peoples, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.9ms
Speed: 1.0ms preprocess, 122.9ms inference, 1.6ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 125.8ms
Speed: 1.0ms preprocess, 125.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.0ms
Speed: 2.0ms preprocess, 122.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 122.

...

Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 2.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.9ms
Speed: 1.1ms preprocess, 124.9ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 125.0ms
Speed: 1.0ms preprocess, 125.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preproc

make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
make wave end
加入殺球:46
加入殺球:55
加入殺球:401
經過殺球判定後
結果 [29, 46, 55, 71, 72, 116, 118, 138, 151, 169, 170, 197, 198, 246, 248, 268, 304, 334, 335, 348, 384, 396, 401]

加入小球3:151,2:5.00469970703125,3:11.991455078125
加入小球1:320,2:3.575408935546875
經過小球判定後
[29, 46, 55, 71, 72, 116, 118, 138, 151, 169, 170, 197, 198, 246, 248, 268, 304, 320, 334, 335, 348, 384, 396, 401]

切球2:76,angle:117
經過切球判定後
結果 [29, 46, 55, 71, 72, 76, 116, 118, 138, 151, 169, 170, 197, 198, 246, 248, 268, 304, 320, 334, 335, 348, 384, 396, 401]

74.04731342315706
221.43728836090094
139.5455709672394
179.53089630711347
176.82584120465435
164.55122375578202
180.83912471127096
160.77626410928
65.23925537219006
132.70488302852465
81.81562202382376
74.93172239214363
98.497971432999


0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 2.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 125.0ms
Speed: 1.0ms preprocess, 125.0ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preprocess, 126.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 2 balls, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 127.0ms
Speed: 1.0ms preprocess, 127.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peop

Speed: 2.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.6ms
Speed: 1.0ms preprocess, 124.6ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 125.0ms
Speed: 1.0ms preprocess, 125.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.8ms
Speed: 1.0ms preprocess, 124.8ms inference, 1.6ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 124.0ms
Speed: 1.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 125.3ms
Speed: 1.0ms preprocess, 125.3ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 736x1280 2 peoples, 1 ball, 126.0ms
Speed: 1.0ms preproc

KeyboardInterrupt: 